In [1]:
import networkx as nx
import random
import numpy as np

def generate_random_graph(num_graph,num_node_min,num_node_max,prob_min,prob_max):
    graphs = []
    for i in range(num_graph):
        num_nodes = random.randint(num_node_min, num_node_max)
        edge_prob = random.uniform(prob_min, prob_max)
        graph = nx.gnp_random_graph(num_nodes, edge_prob)
        graphs.append(graph)
    return graphs

In [2]:
def coloracao_gulosa(G, order):
    color = {}
    classes = []
    for v in order:
        # Cores disponíveis
        available = [True] * G.number_of_nodes()
        for u in G[v]:
            if u in color:
                available[color[u]] = False
        for i in range(G.number_of_nodes()):
            if available[i]:
                color[v] = i
                if i < len(classes):
                    classes[i].append(v)
                else:
                    classes.append([v])
                break
    return classes, color

In [3]:
def two_phase_coloring(G, order):
    # Fase 1: Coloração inicial (greedy)
    order2 = sorted(order, key=lambda v: G.degree(v), reverse=True)
    classes, initial_coloring = coloracao_gulosa(G, order2)
    
    # Fase 2: Coloração na ordem inversa da inicial
    reverse_order = sorted(initial_coloring.keys(), key=lambda x: initial_coloring[x], reverse=True)
    second_classes, second_coloring = coloracao_gulosa(G, reverse_order)
    
    return second_classes

In [4]:
def bb_clique(G, V, clique_atual, best_clique):
    if len(V) == 0:
        if len(clique_atual) > len(best_clique):
            best_clique.clear()
            best_clique.extend(clique_atual)
    else:
        classes = two_phase_coloring(G, list(V))
        limite_superior = len(classes)
        if len(clique_atual) + limite_superior > len(best_clique):
            V = sorted(V, key=lambda v: G.degree(v), reverse=True)
            usado = set([])
            for v in list(V):
                clique_atual.append(v)
                V2 = {u for u in V if u in G[v] and u not in usado}
                if len(V2)+len(clique_atual) > len(best_clique):
                    bb_clique(G, V2, clique_atual, best_clique)
                usado.add(v)
                clique_atual.pop()

In [5]:
import os
import time
from io import StringIO

def find_max_clique(G, arquivo):
    best_clique = []
    estado = []
    V = set(G.nodes())
    V = sorted(V)
    matriz_adj = np.array(nx.adjacency_matrix(G).todense())
    for linha in matriz_adj:
        estado.append("AA " + ', '.join(map(str, linha)) + "\n")
        
    tempo_ini = time.time()
    bb_clique(G, V, [], best_clique)
    tempo_fim = time.time()
    
    estado.append("AA clique arvore " + ' '.join(map(str, best_clique)) + "\n")
    estado.append(f"AA tempo da arvore {tempo_fim-tempo_ini}")
    with open(arquivo,'a') as f:
        f.writelines(estado)

In [ ]:
def main():
    graphs = generate_random_graph(500, 10, 250, 0.3, 0.8)
    i=0
    os.makedirs("instanciasTeste", exist_ok=True)
    for G in graphs:
        find_max_clique(G, f"instanciasTeste/{i}.txt")
        i+=1

if __name__ == "__main__":
    main()

362
363
364
